In [ ]:
#|default_exp concurrent.remote.test_RemoteController_0

In [ ]:
#|export
import asyncio, os, sys, subprocess
from multiprocessing.connection import Listener
from fbdev._utils import TaskManager, find_available_port
from fbdev.dev_utils import is_in_repl
from fbdev.concurrent.remote import RemoteController, get_client

In [ ]:
#|export
port_num = find_available_port()

remote_py_script = """
address = ('localhost', %s)
authkey = b'secret'

import asyncio, os, multiprocessing
from multiprocessing.connection import Listener
from fbdev._utils import TaskManager
from fbdev.concurrent.remote import RemoteController

listener = Listener(address, authkey=authkey)
async def main(conn):
    print("Child PID:", os.getpid())
    task_manager = TaskManager(None)
    child_values = {}
    def send_val_to_child(val): child_values['val'] = val
    remote = RemoteController(conn, task_manager,
        main={
            'msg_from_parent' : lambda msg: print(f"Child (PID: {os.getpid()}) received: '{msg}'"),
            'send_val_to_child' : send_val_to_child,
        }
    )
    
    remote.send_ready()
    await remote.await_ready()
    
    await task_manager.exec_coros(
        remote.do('main', 'msg_from_child', 'Hello from child!'),
        remote.do('main', 'send_val_to_parent', 'from_child'),
        asyncio.sleep(0.5),
        sequentially=True
    )
    assert child_values['val'] == 'from_parent'

with listener.accept() as conn:
    asyncio.run(main(conn))
""" % port_num

address = ('localhost', port_num)
authkey = b'secret'

async def main():
    task_manager = TaskManager(None)
    proc = subprocess.Popen([sys.executable, '-c', remote_py_script])

    with get_client(address, authkey=authkey) as conn:
        print("Parent PID:", os.getpid())
        parent_values = {}
        def send_val_to_parent(val): parent_values['val'] = val
        remote = RemoteController(conn, task_manager,
            main={
                'msg_from_child' : lambda msg: print(f"Parent (PID: {os.getpid()}) received: '{msg}'"),
                'send_val_to_parent' : send_val_to_parent,
            }
        )
        
        remote.send_ready()
        await task_manager.exec_coros(
            remote.await_ready(),
            remote.do('main', 'msg_from_parent', 'Hello from parent!'),
            remote.do('main', 'send_val_to_child', 'from_parent'),
            asyncio.sleep(0.5),
            sequentially=True
        )

        assert parent_values['val'] == 'from_child'
        
    await task_manager.destroy()
    proc.communicate()  # This will wait until the process completes
    
if not is_in_repl():
    asyncio.run(main())

In [ ]:
await main()

Parent PID: 37685
Child PID: 37688
Parent (PID: 37685) received: 'Hello from child!'
Child (PID: 37688) received: 'Hello from parent!'
